In [1]:
from copy import deepcopy
import sys
sys.path.insert(0, "../esinet")
from scipy.optimize import minimize
import numpy as np
from esinet.forward import create_forward_model, get_info
from esinet import Simulation, Net
from esinet.util import unpack_fwd
pp = dict(hemi='both', surface='white')

In [2]:
info = get_info("standard_1020")
fwd = create_forward_model(info=info)
leadfield, pos = unpack_fwd(fwd)[1:3]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.1s remaining:    1.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished


In [3]:
settings=dict(duration_of_trial=0.)
sim = Simulation(fwd, info, settings=settings).simulate(1000)

Simulating data based on sparse patches.


100%|██████████| 1000/1000 [00:00<00:00, 29491.25it/s]


source data shape:  (1284, 1) (1284, 1)


100%|██████████| 1000/1000 [00:01<00:00, 870.37it/s]


In [4]:
source = sim.source_data[0]
eeg = sim.eeg_data[0].average()
source.plot(**pp)

Using pyvista 3d backend.

Using control points [1.32247177e-09 2.30085320e-09 8.26452707e-09]


In [5]:
y = source.data
x = eeg.data
import numpy as np
K_inv = np.linalg.pinv(np.dot(leadfield, leadfield.T))
w = np.matmul(K_inv, x)
y_hat = np.matmul(w.T, leadfield)
source_hat = source.copy()
source_hat.data = y_hat.T
source_hat.plot(**pp)

Using control points [1.88534193e-08 2.38008844e-08 5.29754329e-08]


# scipy minimize

In [6]:
to_vector = lambda a: a.flatten()

y = np.concatenate([source.data for source in sim.source_data], axis=-1)
x = np.concatenate([eeg.average().data for eeg in sim.eeg_data], axis=-1)

In [23]:
from scipy.spatial.distance import cosine
mse = lambda x,y: np.mean((x-y)**2)
def optimize(x, y, leadfield, target_shape):
    def fun(K_inv):
        K_inv_mat = K_inv.reshape(*target_shape)
        w_mat = np.matmul(K_inv_mat, x)
        y_hat = np.matmul(w_mat.T, leadfield).T
        # error = cosine(y_hat.flatten(), y.flatten())
        error = mse(y_hat.flatten(), y.flatten())*1e18
        # print(error)
        return error
    return fun

opt = minimize(optimize(x, y, leadfield, K_inv.shape), to_vector(K_inv), options=dict(maxiter=100))
K_inv_opt = opt['x'].reshape(*K_inv.shape)

Traceback (most recent call last):
  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 51, in safe_event
    return fun(*args, **kwargs)
  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 687, in _clean
    self._layered_meshes[hemi]._clean()
  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 196, in _clean
    mapper = self._actor.GetMapper()
AttributeError: 'NoneType' object has no attribute 'GetMapper'
Traceback (most recent call last):
  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 51, in safe_event
    return fun(*args, **kwargs)
  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 687, in _clean
    self._layered_meshes[hemi]._clean()
  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 196, in _clean
    mapper = self._actor.GetMapper()
AttributeError: 'NoneType' object has no at

## plot scipy minimize results

In [26]:
# simulate new sample
settings=dict(duration_of_trial=0., extents=(25, 35), number_of_sources=1)

sim_test = Simulation(fwd, info, settings=settings).simulate(2)
sim_test.source_data[0].plot(**pp)
y = sim_test.source_data[0].data
x = sim_test.eeg_data[0].average().data

# minimum norm
K_inv = np.linalg.pinv(np.dot(leadfield, leadfield.T))
w = np.matmul(K_inv, x)
y_hat = np.matmul(w.T, leadfield)
source_hat = source.copy()
source_hat.data = y_hat.T
source_hat.plot(**pp)

# optimized inverse solution
w = np.matmul(K_inv_opt, x)
y_hat = np.matmul(w.T, leadfield)
source_hat = source.copy()
source_hat.data = y_hat.T
source_hat.plot(**pp)

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<?, ?it/s]


source data shape:  (1284, 1) (1284, 1)


100%|██████████| 2/2 [00:00<00:00, 501.38it/s]

Using control points [0.0000000e+00 0.0000000e+00 1.0483503e-09]


Using control points [1.58812099e-10 2.09237294e-10 5.01467817e-10]
Using control points [1.46255978e-08 1.57952376e-08 2.02801122e-08]


Using control points [1.46255978e-08 1.57952376e-08 2.02801122e-08]
